In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install -q langchain torch accelerate bitsandbytes

In [4]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM


In [35]:
# To load model 
model_id = "Open-Orca/OpenOrca-Platypus2-13B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model_8 = AutoModelForCausalLM.from_pretrained("Open-Orca/OpenOrca-Platypus2-13B", device_map = "auto", load_in_8bit= True)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│   2 model_id = "Open-Orca/OpenOrca-Platypus2-13B"                                                │
│   3                                                                                              │
│   4 tokenizer = AutoTokenizer.from_pretrained(model_id)                                          │
│ ❱ 5 model_8 = AutoModelForCausalLM.from_pretrained("Open-Orca/OpenOrca-Platypus2-13B", devic     │
│   6                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:484 in          │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   481 │   │   │   )                                                                              │
│   482 │   │   elif type(config) in cls._model_mapping.keys():                                    │
│   483 │   │   │   model_class = _get_model_class(config, cls._model_mapping)                     │
│ ❱ 484 │   │   │   return model_class.from_pretrained(                                            │
│   485 │   │   │   │   pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs,   │
│   486 │   │   │   )                                                                              │
│   487 │   │   raise ValueError(                                                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:2675 in from_pretrained   │
│                                                                                                  │
│   2672 │   │   │   init_contexts.append(init_empty_weights())                                    │
│   2673 │   │                                                                                     │
│   2674 │   │   with ContextManagers(init_contexts):                                              │
│ ❱ 2675 │   │   │   model = cls(config, *model_args, **model_kwargs)                              │
│   2676 │   │                                                                                     │
│   2677 │   │   # Check first if we are `from_pt`                                                 │
│   2678 │   │   if use_keep_in_fp32_modules:                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: LlamaForCausalLM.__init__() got an unexpected keyword argument 'max_new_tokens'

In [50]:
model_8bit = model

In [7]:
gpu_info = !nvidia-smi
gpu_info= '\n'.join(gpu_info)
print(gpu_info)

Mon Aug 21 05:00:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    29W /  70W |   7778MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

In [48]:
# to generate text from loaded model
name = "Open-Orca/OpenOrca-Platypus2-13B"
text = "what is the capital of india?"
max_new_tokens = 1000

def generate_from_model(model, tokenizer, text):
  encoded_input = tokenizer(text, return_tensors='pt')
  output_sequences = model.generate(input_ids=encoded_input['input_ids'].cuda())
  return tokenizer.decode(output_sequences[0], skip_special_tokens=True)

In [49]:
print(generate_from_model(model, tokenizer, text))

what is the capital of india?

The capital of India is New Delhi.


In [70]:
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

pipe = pipeline(
    "text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=500
)

local_llm = HuggingFacePipeline(pipeline=pipe)


In [65]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

template = """you are an expert in python languge who returns only the python code for following user query: 
query: {question}"""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [71]:
llm_chain = LLMChain(prompt=prompt, 
                     llm=local_llm
                     )

question = "print a stacked bar chart"
print(llm_chain.run(question))

 with the given data
data: [
    ['Apple', 10],
    ['Orange', 20],
    ['Banana', 30],
    ['Mango', 40],
    ['Pineapple', 50]
]

Here's the Python code to create a stacked bar chart:

import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111)

data = [
    ['Apple', 10],
    ['Orange', 20],
    ['Banana', 30],
    ['Mango', 40],
    ['Pineapple', 50]
]

colors = ['#F7B6D2', '#FDB45C', '#729FDB', '#F781BE', '#008080']

rects = []
for name, count in data:
    rects.append(plt.Rectangle((0, 0), 1, count, name))

for rect in rects:
    ax.add_patch(rect)

plt.show()

In this code, we create a stacked bar chart using matplotlib.pyplot library. The data is provided in the form of a list of lists, with each inner list containing the name of the fruit and its count. The colors are assigned to each bar using the colors list. Finally, the chart is displayed using plt.show().

Please note that this code only provides the Python implementation for creating a stacked bar chart